In [1]:
import sys, glob, os
from commons import convert_to_json, triq_optimization, qiskit_optimization, \
    calibration_type_enum, qiskit_compilation_enum, normalize_counts, calculate_success_rate_tvd, \
    convert_dict_binary_to_int, convert_dict_int_to_binary, sum_last_n_digits_dict
import wrappers.qiskit_wrapper as qiskit_wrapper
from wrappers.qiskit_wrapper import QiskitCircuit
import pandas as pd
import mthree
import mapomatic as mm
import mthree

from qiskit.quantum_info import SparsePauliOp
from qiskit import QuantumCircuit
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit_ibm_runtime.options import SamplerOptions, EstimatorOptions, DynamicalDecouplingOptions, TwirlingOptions

from qiskit_aer import AerSimulator, QasmSimulator, Aer
from qiskit.qasm2 import dumps
import matplotlib.pyplot as plt
import numpy as np

import mitiq
from mitiq import zne, benchmarks

from qEmQUIP import QEM, conf

CB_color_cycle = ['#1F77B4', '#AEC7E8', '#FF7F0E', '#FFBB78', '#2CA02C', '#98DF8A', '#D62728', '#FF9896',
                  '#9467BD', '#C5B0D5', '#8C564B', '#C49C94', '#E377C2', '#F7B6D2', '#7F7F7F', '#C7C7C7',
                  '#BCBD22', '#DBDB8D', '#17BECF', '#9EDAE5'
                  ]

markers = ['o', 'v', '^', 's', '+', '*', 'x', 'd', '<', '>', 'p']
linestyles = ['-', '--', '-.', ':', '-', '--', '-.', ':']



In [2]:
token = "476ea8c61cc54f36e4a21d70a8442f94203c9d87096eaad0886a3e8154d8c2e79bcad6f927c6050a76335dd68d783f478c1b828504748a4377b441c335c831aa"
q = QEM(runs=conf.runs, user_id=conf.user_id, token=token, skip_db=True)

Time for setup the services: 10.93546121899999 seconds


In [3]:
qasm_files = q.get_qasm_files_from_path("./circuits/polar_sim/")
print(qasm_files)

# select compilation techniques
compilations = ["qiskit_3", "triq_avg_sabre", "triq_lcd_sabre"]

['./circuits/polar_sim/polar_all_meas_n3_x.qasm', './circuits/polar_sim/polar_all_meas_n3.qasm']


In [ ]:
res_circuit_name = []
res_compilations = []
res_noise_levels = []
res_success_rate = []
res_success_rate_dd = []
res_success_rate_m3 = []
res_success_rate_dd_m3 = []

# Prepare the object
q = QEM(runs=conf.runs, user_id=conf.user_id, token=token)

# prepare the backend
q.set_backend(program_type="sampler")

# prepare the circuit
qasm_files = q.get_qasm_files_from_path("./circuits/polar_sim/")

shots = 100

# select compilation techniques
compilations = ["qiskit_3", "triq_avg_sabre", "triq_lcd_sabre"]
# compilations = ["qiskit_0", "qiskit_3"]

# select the noise levels (0 = noiseless, 1 = noise model from real backend)
noise_levels = [0.0, 0.1, 0.2, 0.4, 0.6, 0.8, 1.0]
#noise_levels = [0.0]


for qasm in qasm_files:
    qc = q.get_circuit_properties(qasm_source=qasm)
    circuit_name = qasm.split("/")[-1].split(".")[0]

    for comp in compilations:    
        updated_qasm, initial_mapping = q.compile(qasm=qc.qasm_original, compilation_name=comp)
        compiled_qc = QiskitCircuit(updated_qasm)
        circuit = compiled_qc.transpile_to_target_backend(q.backend)

        print("Circuit : ", circuit_name, ", Compilation : ", comp)
        
        for noise in noise_levels:
            
            noise_model, noisy_simulator, coupling_map = qiskit_wrapper.get_noisy_simulator(q.backend, noise)
            
            job = noisy_simulator.run(circuit, shots=shots)
            result = job.result()  
            output = result.get_counts()
            output_normalize = normalize_counts(output, shots=shots)

            tvd = calculate_success_rate_tvd(qc.correct_output,output_normalize)

            probs_m3 = q.apply_mthree(noisy_simulator, initial_mapping, output, shots)
            tvd_m3 = calculate_success_rate_tvd(qc.correct_output, probs_m3)
            
            circuit_dd = q.apply_dd(circuit, noisy_simulator, sequence_type="XX", scheduling_method="asap")
            job_dd = noisy_simulator.run(circuit_dd, shots=shots)
            result_dd = job_dd.result()  
            output_dd = result.get_counts()
            output_normalize_dd = normalize_counts(output_dd, shots=shots)

            tvd_dd = calculate_success_rate_tvd(qc.correct_output,output_normalize)

            probs_dd_m3 = q.apply_mthree(noisy_simulator, initial_mapping, output_dd, shots)
            tvd_dd_m3 = calculate_success_rate_tvd(qc.correct_output, probs_m3)
            
            
            print("Noise level :", noise, "- sr :", np.round(tvd,4), ", sr m3:", np.round(tvd_m3,4),
                  ", sr dd :", np.round(tvd_dd,4), ", sr dd m3:", np.round(tvd_dd_m3,4))
            res_circuit_name.append(circuit_name)
            res_compilations.append(comp)
            res_noise_levels.append(noise)
            res_success_rate.append(tvd)
            res_success_rate_m3.append(tvd_m3)
            res_success_rate_dd.append(tvd_dd)
            res_success_rate_dd_m3.append(tvd_dd_m3)

        


Time for setup the services: 21.224898121996375 seconds
Time for setup the backends: 7.412499689962715e-05 seconds
Circuit :  polar_all_meas_n3_x , Compilation :  qiskit_3
Noise level : 0.0 - sr : 0.5123 , sr m3: 0.5007 , sr dd : 0.5123 , sr dd m3: 0.5007
Noise level : 0.1 - sr : 0.5005 , sr m3: 0.5 , sr dd : 0.5005 , sr dd m3: 0.5
Noise level : 0.2 - sr : 0.5 , sr m3: 0.5001 , sr dd : 0.5 , sr dd m3: 0.5001
Noise level : 0.4 - sr : 0.5 , sr m3: 0.5002 , sr dd : 0.5 , sr dd m3: 0.5002
Noise level : 0.6 - sr : 0.5 , sr m3: 0.5 , sr dd : 0.5 , sr dd m3: 0.5
Noise level : 0.8 - sr : 0.5 , sr m3: 0.5 , sr dd : 0.5 , sr dd m3: 0.5
Noise level : 1.0 - sr : 0.5 , sr m3: 0.5 , sr dd : 0.5 , sr dd m3: 0.5
Circuit :  polar_all_meas_n3_x , Compilation :  triq_avg_sabre
Noise level : 0.0 - sr : 0.5 , sr m3: 0.5 , sr dd : 0.5 , sr dd m3: 0.5
Noise level : 0.1 - sr : 0.5 , sr m3: 0.5 , sr dd : 0.5 , sr dd m3: 0.5
Noise level : 0.2 - sr : 0.5 , sr m3: 0.5 , sr dd : 0.5 , sr dd m3: 0.5
Noise level : 0